# Extracción de datos web (_Web scrapping_)  

Ante la generación masiva a traves de la red es importante tener herramientas que permitan la extracción de datos a partir de fuentes cuya ubicación es esta. De esto se trata el _web scrapping_.  

Se pueden tener elementos poco especificos mediante las mismas alternativas del procesamiento de texto para algunos casos, sin embargo esto no siempre será efectivo ni eficiente. Por ejemplo, podemos usar `wget` para descargar una página y hacer la búsqueda de elementos `html` en ella por medio de expresiones regulares, pero la descarga de la página implica que el contenido debio ser estatico. Igualmente, las expresiones regulares no son la mejor herramienta siempre, y es más eficiente usar elementos especialmente diseñados para recorrer la estructura html sin depender de la generación de expresiones de coincidencia sino obedeciendo exclusivamente a los patrones que ya sabemos que existirán por defecto.  

## Herramientas (en python)

Para esta labor contamos con algunas herramientas como lo son:  

+ `urllib`: Modulo incluido en python para la recuperación de contenido de una url.  
+ `webbrowser`: Modulo incluido en python para la apertura de url's en una instancia del navegador predefinido.  
+ `html`: Modulo incluido en python para el analisis sintactico html.  
+ Request: Reemplazo externo para `urllib` con mayores caracteristicas.  
+ Beautiful Soup: Reemplazo externo para `html` con mayores caracteristicas.  
+ Selenium: Reemplazo externo para `webbrowser` con mayores caracteristicas.  
+ Wget: _Port_ de `wget` para python.  

## Instalar requisitos

Primero que todo, partimos que ya tenemos instalado al menos un navegador (firefox por defecto en la mayor parte de las distribuciones linux). Se puede trabajar con otros navegadores, y es de especial interes PhantomJS, una opción de navegador que no genera interface gráfica, ideal para pruebas o automatización (en caso de ser molesto que el navegador se vea abrir y cerrar, etc...).  

    pip install selenium beautifulsoup4 Requests

## Aplicando

In [1]:
import webbrowser

Al usar la función `open` de `webbrowser` se abrirá el navegador o una pestaña nueva si el navegador ya estaba abierto. Ya que no indicamos el navegador, esto se realiza con el navegador configurado por defecto en nuestro sistema. Se puede usar para abrir pestañas y ventanas nuevas, cerrarlo, y tambien usar un navegador especifico.

In [2]:
webbrowser.open('http://github.com/')

True

Sin embargo, la labor de extracción web depende de obtener el código fuente o elementos disponibles en las páginas, lo cual es imposible con solo abrir el navegador. Para este fin es posible usar `urllib` o como lo haremos en esta sesión, con `request`.

In [3]:
import requests
res = requests.get('http://www.gutenberg.org/files/18251/18251-0.txt')
res.status_code == requests.codes.ok # Validar código 200 (ok)
type(res)
len(res.text)
print(res.text[:250])

﻿Project Gutenberg's Latin for Beginners, by Benjamin Leonard D'Ooge

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project G


Ante un fallo en el proceso de obtención del código con la función `get`, es posible generar una notificación del motivo de fallo.

In [4]:
res = requests.get('http://github.com/yomeinventoesto')
res.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://github.com/yomeinventoesto

Cuando obtenemos un elemento de una dirección, este se encuentra como binario y no como texto plano. Esto nos facilita algunas cosas. Nos permite descargar contenido que no se solo texto plano (archivos de texto o código fuente) sino tambien directamente archivos binarios como imagenes, ejecutables, videos, archivos de word y otros. Es importante aclarar, que si vamos a almacenar el archivo de texto plano, debemos hacerlo con creación de archivos binarios para no perder la codificación original que tenga el archivo.

In [5]:
res = requests.get('http://www.programmableweb.com/sites/default/files/github-jupyter.jpg')
archivo_imagen = open('github-jupyter.jpg', 'wb')
for bloques in res.iter_content(100000):
    archivo_imagen.write(bloques)
archivo_imagen.close()

![github jupyter](github-jupyter.jpg)

En el bloque anterior, el método `iter_content` genera bloques del archivo con el tamaño indicado en su argumento. Esto conviene para la escritura de archivos de gran tamaño.

In [6]:
import bs4

Usarmos ahora `bs4` (forma de importar Beautiful Soup), lo cual nos permitirá la búsqueda de texto y estructuras html especificas. Este es más conveniente que usar expresiones regulares directamente en el código fuente.  

Al crear el objeto, debemos indicar el texto sobre el cual actuará (puede ser obtenido directamente de un archivo abierto tambien) y el tipo de analizador sintactico, en este caso `lxml`.

In [7]:
res = requests.get('https://github.com/cosmoscalibur/herramientas_computacionales')
gh = bs4.BeautifulSoup(res.text, "lxml")
type(gh)

bs4.BeautifulSoup

Ahora, buscaremos todas las estructuras `td` que tengan el atributo `class` con valor `content`.

In [8]:
tabla_archivos = gh.find_all('td', {'class':'content'})
type(tabla_archivos)

bs4.element.ResultSet

El resultado es una lista con todos los resultados obtenidos. Tambien es posible una búsqueda uno a uno, usando `find` en lugar de `find_all`.

In [9]:
len(tabla_archivos)

23

In [10]:
print(tabla_archivos)

[<td class="content" colspan="3">Failed to load latest commit information.</td>, <td class="content">
<span class="css-truncate css-truncate-target"><a class="js-navigation-open" href="/cosmoscalibur/herramientas_computacionales/tree/master/Evaluaci%C3%B3n" id="f65268c13fd62c5d4e9269ebcd9e11ca-8ed6e5c0b159f72f1829ff55eb7c4856e58e43b9" title="Evaluación">Evaluación</a></span>
</td>, <td class="content">
<span class="css-truncate css-truncate-target"><a class="js-navigation-open" href="/cosmoscalibur/herramientas_computacionales/tree/master/Proyecto" id="c126d9cdd553787e63d4a48608f608cc-5099d554c9d22731a50e7784cee7e458bcc9b81f" title="Proyecto">Proyecto</a></span>
</td>, <td class="content">
<span class="css-truncate css-truncate-target"><a class="js-navigation-open" href="/cosmoscalibur/herramientas_computacionales/blob/master/.gitignore" id="a084b794bc0759e7a6b77810e01874f2-8cec0970c2bcc6a64ca702e6ace3072bec1db708" title=".gitignore">.gitignore</a></span>
</td>, <td class="content">
<s

En el filtrado anterior, ahora buscaremos todas las etiquetas `a` las cuales asociamos con la presencia del atributo `href`. De esta forma localizaremos la lista de archivos. Para obtener el texto al interior de una etiqueta, usamos la propiedad `string` y el valor de un atributo con el método `get`.

In [11]:
for content in tabla_archivos:
    lineas_a = content('a')
    if lineas_a:
        texto = "Se encontro el archivo '{}'".format(lineas_a[0].string.encode("utf-8"))
        texto += " con enlace '{}'.".format(lineas_a[0].get("href"))
        print(texto)

Se encontro el archivo 'b'Evaluaci\xc3\xb3n'' con enlace '/cosmoscalibur/herramientas_computacionales/tree/master/Evaluaci%C3%B3n'.
Se encontro el archivo 'b'Proyecto'' con enlace '/cosmoscalibur/herramientas_computacionales/tree/master/Proyecto'.
Se encontro el archivo 'b'.gitignore'' con enlace '/cosmoscalibur/herramientas_computacionales/blob/master/.gitignore'.
Se encontro el archivo 'b'Git.ipynb'' con enlace '/cosmoscalibur/herramientas_computacionales/blob/master/Git.ipynb'.
Se encontro el archivo 'b'Jupyter Notebook Basico.ipynb'' con enlace '/cosmoscalibur/herramientas_computacionales/blob/master/Jupyter%20Notebook%20Basico.ipynb'.
Se encontro el archivo 'b'Jupyter Notebook Intermedio.ipynb'' con enlace '/cosmoscalibur/herramientas_computacionales/blob/master/Jupyter%20Notebook%20Intermedio.ipynb'.
Se encontro el archivo 'b'LICENSE'' con enlace '/cosmoscalibur/herramientas_computacionales/blob/master/LICENSE'.
Se encontro el archivo 'b'LaTeX_basico.pdf'' con enlace '/cosmoscali

Nos vimos en la necesidad de usar `encode("utf-8")` ya que la codificación de la página es utf-8 y no ascii (el usado por defecto en python). Podemos consultar los atributos de una etiqueta o si posee un atributo especifico, y no solo obtener el valor, de la siguiente forma.

In [12]:
lineas_a[0].has_attr("href") # Existencia de un atributo

True

In [13]:
lineas_a[0].attrs # Atributos existentes

{'class': ['js-navigation-open'],
 'href': '/cosmoscalibur/herramientas_computacionales/blob/master/web_scrapping.ipynb',
 'id': '634ac137a7e35d0e03e15bc603462ab7-983a57f7dab52e377388e0e5e2a7df7750a7dde4',
 'title': 'web_scrapping.ipynb'}

In [14]:
from selenium import webdriver

Invocar la instancia del controlador del navegador depende del navegador de interes. Hay que tener encuenta que no todos los navegadores son soportados. Podemos encontrar soporte para Chrome, Firefox, Opera, IE y PhantomJS. Este último permite realizar la labor sin la generación de una ventana para el navegador (en caso de ser necesario, incluso se puede generar capturas de pantalla para su validación con ayuda del controlador).  
Acorde a cada navegador, se puede tener requerimientos especificos. En el caso de firefox, se presenta la necesidad de indicar el directorio del perfil de usuario, en el caso de chrome se requiere indicar la ruta del controlador (se [descarga](https://sites.google.com/a/chromium.org/chromedriver/) ya que no viene incluido como si sucede en firefox o phantomjs).  
Podría ser posible (no he verificado) usar otros navegadores si usan el mismo motor de navegación realizando la indicación explicita de la ruta del ejecutable. Por ejemplo, se podría controlar vivaldi realizando el cambio de ruta de chrome (usan el mismo motor de navegación).

In [15]:
browser = webdriver.Chrome("/home/cosmoscalibur/Downloads/chromedriver")
browser.get('http://github.com')
username = browser.find_element_by_id("user[login]")
username.send_keys("cosmoscalibur@gmail.com")
dar_click = browser.find_element_by_link_text("privacy policy")
dar_click.click()

Resulta bastante útil el uso de `selenium` no tanto en los casos que requieran de interacción sino en los casos donde los contenidos (incluye elementos de interacción) son de generación dinámica o tras la interacción el nuevo enlace o contenido tiene retrasos apreciables, lo cual evitaría que `Request` obtenga el código adecuado. Podemos extraer el código fuente de la página en la cual se encuentra el foco del navegador de la siguiente forma.

In [16]:
codigo = browser.page_source
print(codigo)

<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" class=" is-copy-enabled is-u2f-enabled"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# object: http://ogp.me/ns/object# article: http://ogp.me/ns/article# profile: http://ogp.me/ns/profile#">
    <meta charset="utf-8" />
    <meta content="origin-when-cross-origin" name="referrer" />

    <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-cb37473586c0bff2206ff7c864d9afda5e2063afb40364d87d64eefa2536d1c0.css" integrity="sha256-yzdHNYbAv/Igb/fIZNmv2l4gY6+0A2TYfWTu+iU20cA=" media="all" rel="stylesheet" />
    <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-1d1dd698ec7e26200c41f689c19707b13992a803afc430c1abe93cc850c026a7.css" integrity="sha256-HR3WmOx+JiAMQfaJwZcHsTmSqAOvxDDBq+k8yFDAJqc=" media="all" rel="stylesheet" />
    
    
    <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/site-b637b3b72afffd79585a758c94c7bd5bc8d4

Si se desea hacer pasar algún navegador especifico o una solicitud por `request` o `urllib` como un navegador dado (por ejemplo, evitar bloqueos de contenido por navegador o por politicas contra la extracción web), es necesario realizar la modificación del `user-agent`.

## Bibliografía

+ [How to fetch Internet Resources Using The urllib Package](https://docs.python.org/3/howto/urllib2.html).  
+ [Structured Markup Processing Tools](Structured Markup Processing Tools).  
+ [Internet Protocols and Support](https://docs.python.org/3/library/internet.html).  
+ Automate the boring stuff. [Chapter 11 – Web Scraping](https://automatetheboringstuff.com/chapter11/).  
+ [First web scraper](https://first-web-scraper.readthedocs.io/en/latest/#act-3-web-scraping).  
+ [HOW TO DOWNLOAD DYNAMICALLY LOADED CONTENT USING PYTHON](https://dvenkatsagar.github.io/tutorials/python/2015/10/26/ddlv/).  